In [1]:
# from predset_revised import MahjongSLpredset
from dataset import MahjongSLDataset
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn.functional as F
import os
import torch
from torch import nn

# from model import predEntryModule
from collections import defaultdict

# from model_batched_MNN import MahJongNetBatchedRevised, TileProbModuleNew
import feature
import matplotlib.pyplot as plt
import json
import rule

# from fanCalcLib import formMinComb_c
import copy


def default_zero():
    return 0


def default_list():
    return []


def default_one():
    return 1


def default_three_zeros():
    return [0, 0, 0]


plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["figure.figsize"] = (15, 7)
plt.rcParams["font.size"] = 16

# fmt:off
tile_list_raw = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9',  # 饼
                 'W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9',  # 万
                 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9',  # 条
                 'F1', 'F2', 'F3', 'F4', 'J1', 'J2', 'J3'  # 风、箭
                 ]

tile_list_raw_cn = ["一饼","二饼","三饼","四饼","五饼","六饼","七饼","八饼","九饼",  # 饼
                 "一万","二万","三万","四万","五万","六万","七万","八万","九万",  # 万
                 "一条", "二条", "三条", "四条", "五条", "六条", "七条", "八条", "九条",   # 条
                 "东风", "南风","西风","北风", '中', '发', '白'  # 风、箭
                 ]
fan_list_raw = ["大四喜", "大三元", "绿一色", "九莲宝灯", "四杠", "连七对", "十三幺", "清幺九", "小四喜", "小三元", "字一色", "四暗刻", "一色双龙会",
                "一色四同顺", "一色四节高", "一色四步高", "三杠", "混幺九", "七对", "七星不靠", "全双刻", "清一色", "一色三同顺", "一色三节高", "全大",
                "全中", "全小", "清龙", "三色双龙会", "一色三步高", "全带五", "三同刻", "三暗刻", "全不靠", "组合龙", "大于五", "小于五", "三风刻", "花龙",
                "推不倒", "三色三同顺", "三色三节高", "无番和", "妙手回春", "海底捞月", "杠上开花", "抢杠和",
                "碰碰和", "混一色", "三色三步高", "五门齐", "全求人", "双暗杠", "双箭刻", "全带幺",
                "不求人", "双明杠", "和绝张", "箭刻", "圈风刻", "门风刻", "门前清", "平和", "四归一",
                "双同刻", "双暗刻", "暗杠", "断幺", "一般高", "喜相逢", "连六", "老少副", "幺九刻",
                "明杠", "缺一门", "无字", "边张", "嵌张", "单钓将", "自摸"]

fan_list_raw_en = ["Big Four Winds", "Big Three Dragons", "All Green", "Nine Gates", "Four Kongs", "Seven Shifted Pairs", "Thirteen Orphans", "All Terminals", "Little Four Winds", "Little Three Dragons",
                   "All Honor", "Four Concealed Pungs", "Pure Terminal Chows", "Quadruple Chow", "Four Pure Shifted Pungs", "Four Pure Shifted Chows", "Three Kongs", "All Terminals and Honors", 
                   "Seven Pairs", "Greater Honors and Knitted Tiles", "All Even Pungs", "Full Flush", "Pure Triple Chow", "Pure Shifted Pungs", "Upper Tiles", "Middle Tiles", "Lower Tiles","Pure Straight",
                   "Three-Suited Terminal Chows", "Pure Shifted Chows", "All Fives", "Triple Pung", "Three Concealed Pungs", "Lesser Honors and Knitted Tiles", "Knitted Straight", "Upper Five", "Lower Five",
                   "Big Three Winds", "Mixed Straight", "Reversible Tiles", "Mixed Triple Chow", "Mixed Shifted Pungs", "Chicken Hand", "Last Tile Draw", "Last Tile Claim", "Out with Replacement Tile", "Robbing The Kong",
                    "All Pungs", "Half Flush", "Mixed Shifted Chows", "All Types", "Melded Hand","Two Concealed Kongs", "Two Dragons Pungs", "Outside Hand", "Fully Concealed Hand", "Two Melded Kongs", "Last Tile", 
                   "Dragon Pung", "Prevalent Wind", "Seat Wind", "Concealed Hand", "All Chows", "Tile Hog", "Double Pung", "Two Concealed Pungs", "Concealed Kong", "All Simples", "Pure Double Chow", "Mixed Double Chow",
                   "Short Straight", "Two Terminal Chows", "Pung of Terminals or Honors", "Melded Kong", "One Voided Suit", "No Honors", "Edge Wait", "Closed Wait", "Single Waiting", "Self-Drawn"
]

tile_list_conversion = {'B1':'D1', 'B2':'D2', 'B3':'D3', 'B4':'D4', 'B5':'D5', 'B6':'D6', 'B7':'D7', 'B8':'D8', 'B9':'D9',  # 饼
                 'W1':'C1', 'W2':'C2', 'W3':'C3', 'W4':'C4', 'W5':'C5', 'W6':'C6', 'W7':'C7', 'W8':'C8', 'W9':'C9',  # 万
                 'T1':'B1', 'T2':'B2', 'T3':'B3', 'T4':'B4', 'T5':'B5', 'T6':'B6', 'T7':'B7', 'T8':'B8', 'T9':'B9',  # 条
                 'F1':'EW', 'F2':'SW', 'F3':'WW', 'F4':'NW', 'J1':'RD', 'J2':'GD', 'J3':'WD'  # 风、箭
                 }
# fmt:on

In [2]:
tile_list_raw.index("T9")

26

In [3]:
id = fan_list_raw.index("双明杠")
print(id)

56


In [4]:
fan_list_raw_en[id]

'Two Melded Kongs'

In [5]:
# change params here
log_name = "07_16_02_53_57-model_slim_rev6_normed2"

# typically leave as is
logdir = "../log/"
log_suffix = "checkpoint"
extract_dir = "extracted_model_slim_params"
weight_list = {
    "top1": "best_acc.pkl",
    "top2": "best_acc_top2.pkl",
    "top3": "best_acc_top3.pkl",
}

In [14]:
class TileProbModuleNew(nn.Module):
    """
    Computation for opponent_desire_prob, held_prob, discard_prob
    """

    def __init__(self):
        super(TileProbModuleNew, self).__init__()

        self.final_layer = nn.Sequential(
            nn.Linear(11, 1),
        )

    def forward(self, meta_feature, tile_wall_feature):
        # input: meta (data[:4])

        meta_feature_expanded = meta_feature.view(-1, 1, 6).repeat(1, 34, 1).view(-1, 6)
        tile_wall_feature_flatten = tile_wall_feature.view(-1, 5)
        concated_feature = torch.cat(
            (meta_feature_expanded, tile_wall_feature_flatten), dim=1
        )
        # print(concated_feature.shape)

        final = self.final_layer(concated_feature)
        return final.view(-1, 34)


def reversed_tile_conversion(list_rep, key_order_list):
    ret_dict = defaultdict(default_zero)
    for i in range(len(key_order_list)):
        ret_dict[key_order_list[i]] += list_rep[i]
    return ret_dict


class MahJongNetBatchedRevised(nn.Module):
    """
    This module computes data_prep ([32*4, 80, 32, 2]) object
    """

    def __init__(self, device="cpu"):
        super(MahJongNetBatchedRevised, self).__init__()
        self.device = device
        self.fan_coeff = nn.Parameter(
            torch.tensor(
                [0.1 + torch.randn(1) * 0.015 for _ in range(80)],
                requires_grad=True,
            )
        )
        self.fan_coeff_multiplier = torch.tensor(
            [1.0 for _ in range(80)], device=device
        )
        self.fan_coeff_multiplier[33] *= 21
        self.tile_coeff = nn.Parameter(
            torch.tensor(
                [1.0 + torch.randn(1) * 0.015 for _ in range(34)], requires_grad=True
            )
        )
        # self.QQR_pack_penalty = nn.Parameter(torch.tensor(0.1, requires_grad=True))

        self.prob_module_throw = TileProbModuleNew().to(device)

    def access_named_params(self):
        return (
            self.fan_coeff.detach().numpy(),
            self.tile_coeff.detach().numpy(),
            # self.QQR_pack_penalty.item(),
        )

    def access_fan(self):
        return self.fan_coeff

    def forward(self, x):
        """
        @data_entry: n * 5 * 32 * (34*4, 80, 34, 2)
        @held_prob: n * 5 * 32 * (34)
        @discard_prob: n * 5 * 32 * (34)
        """

        (meta_feature, tile_wall_feature, search_matrix) = x

        prob_throw = self.prob_module_throw(meta_feature, tile_wall_feature)

        (
            tile_prep_data,
            fan_prep,
            missing_tile_prep_data,
            count_prep,
            chi_peng_count_remain_data,
        ) = search_matrix

        missing_tile_prep = missing_tile_prep_data.view(-1, 320, 34)
        chi_peng_count_remain = chi_peng_count_remain_data.view(-1, 320)
        tile_prep = tile_prep_data.view(-1, 320, 34, 7)
        tile_self_draw_baseline = tile_prep[:, :, :, 0]
        tile_need_count = tile_prep[:, :, :, 1]
        tile_cp_coeff = tile_prep[:, :, :, 2]
        tile_cp_base = tile_prep[:, :, :, 3]
        tile_hidden_ct = tile_prep[:, :, :, 4]
        tile_cp_switch = tile_prep[:, :, :, 5]
        tile_null_filler = tile_prep[:, :, :, 6]

        fan_prep_prime = 1 - fan_prep

        # Throw operation
        # fan preference
        fan_pref = fan_prep * self.fan_coeff * self.fan_coeff_multiplier
        multiplied_fan_pref_throw = torch.sum(fan_pref, dim=-1).view(
            -1, 320
        )  # n * 5 * 64

        # tile probability (throw)
        term1_throw = (
            tile_self_draw_baseline * prob_throw.unsqueeze(1) ** tile_need_count
        )
        term2_throw = tile_cp_base * tile_cp_coeff / tile_hidden_ct  #
        term2_throw_zeroed = torch.nan_to_num(term2_throw)
        term3_throw = tile_null_filler
        terms_summed_throw = (
            term1_throw + term3_throw + term2_throw_zeroed * tile_cp_switch
        )

        multiplied_sum_throw = (
            torch.prod(terms_summed_throw, -1) * 100 * multiplied_fan_pref_throw
        )
        final_prob_throw = (
            multiplied_sum_throw  # * self.QQR_pack_penalty**chi_peng_count_remain
        )

        # distinguish by missing tile (Throw)
        missing_tile_prob_throw = missing_tile_prep * final_prob_throw.unsqueeze(-1)
        # missing_tile_prob_throw += tile_pref_from_fan

        # combine 64 sub entries
        # win-rate transfered to tile, judge tile by tile-bounded win-rate
        missing_tile_prob_throw = missing_tile_prob_throw.view(-1, 5, 64, 34)
        tile_bound_winrate_throw = missing_tile_prob_throw.sum(dim=-2)

        # final tile-bounded prob
        weighted_tile_prob = (tile_bound_winrate_throw * self.tile_coeff)[:, 0]

        # combine 64 sub entries
        # reshape
        # final_prob: win rate for each 64 entries
        final_prob_ops = final_prob_throw.view(-1, 5, 64)
        # win-rate transfered to tile, judge tile by tile-bounded win-rate
        missing_tile_prob_ops = missing_tile_prob_throw
        tile_bound_winrate_ops = missing_tile_prob_ops.sum(dim=2)
        final_prob_ops = final_prob_ops.sum(dim=-1)

        # produce result
        # final tile-bounded prob
        max_tile_prob_for_action, e = torch.max(tile_bound_winrate_ops, dim=2)
        max_tile_prob_for_action[:, 0] = final_prob_ops[:, 0]
        # max_tile_prob_for_action *= self.chi_peng_coeff

        # return (
        #     max_tile_prob_for_action,
        #     weighted_tile_prob,
        # )

        print(
            missing_tile_prob_throw.shape,
            fan_prep.shape,
            missing_tile_prep.shape,
            tile_need_count.shape,
        )

        return (
            missing_tile_prob_throw[0, 0],
            fan_prep,
            missing_tile_prep.view(5, 64, 34),
            tile_need_count.view(5, 64, 34),
            tile_bound_winrate_throw[:,0,:]
        )

        # final tile-bounded prob
        weighted_tile_prob = tile_bound_winrate_throw  # * self.tile_coeff  # - opponent_desire_prob.unsqueeze(1)

        return weighted_tile_prob[:, 0, :]

In [15]:
device = "cpu"
net = MahJongNetBatchedRevised(device).to(device)
# load model parameter from log
net.load_state_dict(
    torch.load(
        os.path.join(logdir, log_name, log_suffix, weight_list["top1"]),
        map_location=torch.device(device),
    )
)

<All keys matched successfully>

In [16]:
path_to_ds = "../tmp"
ds = MahjongSLDataset(path_to_ds, 0.0, 1)
loader = DataLoader(dataset=ds, batch_size=1, shuffle=False, num_workers=4)
d = next(iter(loader))
d[1] == ds[0][1]

tensor([[True, True, True, True, True, True]])

In [17]:
(
    meta,
    meta_feature_new,
    tile_wall_feature,
    k,
    q,
    m,
    n,
    o,
    # fan_summary,
    label_action,
    label_tile,
    v_info,
) = d

(
    meta,
    meta_feature_new,
    tile_wall_feature,
    k,
    q,
    m,
    n,
    o,
    # fan_summary,
    label_action,
    label_tile,
) = (
    meta.to(device),
    meta_feature_new.to(device),
    tile_wall_feature.to(device),
    k.to(device),
    q.to(device),
    m.to(device),
    n.to(device),
    o.to(device),
    # data_description.to(device).float(),
    # fan_summary.to(device),
    label_action.to(device),
    label_tile.to(device),
)
data1 = (k, q, m, n, o)

index_list_plus_one = torch.tensor([i + 1 for i in range(len(meta))]).to(device)
x = (
    meta_feature_new,
    tile_wall_feature,
    data1,
)

In [18]:
meta_feature_new.view(-1).size()+tile_wall_feature.view(-1).size()+k.view(-1).size()+q.view(-1).size()+m.view(-1).size()+n.view(-1).size()+o.view(-1).size()

torch.Size([6, 170, 76160, 25600, 10880, 640, 320])

In [19]:
sum([6, 170, 76160, 25600, 10880, 640, 320])

113776

In [21]:
explosion, fan, throw, missing, tile = net(x)
explosion *= 1 / torch.max(explosion)

torch.Size([1, 5, 64, 34]) torch.Size([1, 5, 64, 80]) torch.Size([1, 320, 34]) torch.Size([1, 320, 34])


In [31]:
a=F.softmax(tile, dim=1)

In [33]:
torch.log(a)

tensor([[-6.2190, -4.9818, -6.2190, -6.2190, -6.2190, -6.2190, -2.4513, -6.2190,
         -6.2190, -6.2190, -6.2190, -4.9818, -6.2190, -6.2190, -4.9818, -6.2190,
         -6.2190, -1.4934, -5.3885, -6.2190, -6.2190, -6.2190, -6.2190, -1.8628,
         -6.2190, -2.3863, -1.0779, -6.2190, -6.2190, -6.2190, -4.7058, -4.5095,
         -6.2190, -4.3659]], grad_fn=<LogBackward0>)

In [11]:
# Generate pandas dataframe
out = explosion.detach().numpy()
df = pd.DataFrame(out, columns=tile_list_raw)
df.style.highlight_max(axis=0)
df = df.round(4)
# Fan list
fan_list = []
for sample in fan[0, 0]:
    fan_list.append([fan_list_raw[i] for i in range(80) if sample[i] == 1])
df["fan"] = fan_list
# Redundant/useless tile
throw_list = []
for sample in throw[0]:
    throw_list.append([tile_list_raw[i] for i in range(34) if sample[i] == 1])
df["discard_choice"] = throw_list
# Tiles in need
tile_list = []
for sample in missing[0]:
    tmp_list = []
    for i in range(34):
        for _ in range(int(sample[i])):
            tmp_list.append(tile_list_raw[i])
    tile_list.append(tmp_list)
df["missing"] = tile_list
# target
# target_list = []
# for i in range(64):
#     hand_list = []
#     if len(tile_list[i]) != 0 and len(throw_list[i]) != 0:
#         hand_dict = handWall_log[line_id - 1][player_id]
#         hand_list = [k for k in hand_dict.keys() for _ in range(hand_dict[k])]
#         for t in tile_list[i]:
#             hand_list.append(t)
#         for t in throw_list[i]:
#             hand_list.remove(t)
#     target_list.append(copy.deepcopy(hand_list))
# df["target"] = target_list

In [12]:
for key in tile_list_raw:
    if sum(df[key]) == 0:
        df.drop(key, axis=1, inplace=True)

In [27]:
# Generate latex table for appendix
df = pd.DataFrame()
df["List of Fans"] = [
    ", ".join(
        [
            fan_list_raw_en[fan_list_raw.index(fan)]
            for fan in fan_ent
            if fan_list_raw.index(fan) < 58
        ]
    )
    for fan_ent in fan_list
]
df["Prob"] = [max(x) for x in out]
df["B9"] = out[:, 26]
df["Count"] = [len(x) for x in throw_list]
df["Missing Tiles"] = [", ".join([tile_list_conversion[i] for i in x]) for x in tile_list]
df["Redundant Tiles"] = [", ".join([tile_list_conversion[i] for i in x]) for x in throw_list]
df = df.round(4)

In [30]:
with open("a.txt", "w+") as f:
    f.writelines(
        df.to_latex(
            index=True,
            formatters={"name": str.upper},
            float_format="{:.4f}".format,
        )
    )

/tmp/ipykernel_3517/2143978880.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(


In [28]:
print(
    df.to_latex(
        index=True,
        formatters={"name": str.upper},
        float_format="{:.4f}".format,
    )
)

\begin{tabular}{llrrrll}
\toprule
{} &                          List of Fans &   Prob &     B9 &  Count &               Missing Tiles &         Redundant Tiles \\
\midrule
0  &                      Knitted Straight & 0.1246 & 0.1246 &      5 &          D5, D7, B4, B7, B7 &      C9, B9, NW, RD, WD \\
1  &                             All Types & 0.0043 & 0.0043 &      6 &      D6, B7, NW, NW, RD, RD &  D2, C3, C6, B1, B9, WD \\
2  &                             All Types & 0.0043 & 0.0043 &      6 &      D6, B7, NW, NW, WD, WD &  D2, C3, C6, B1, B9, RD \\
3  &                             All Types & 0.0044 & 0.0000 &      6 &      D6, B7, NW, NW, RD, RD &  D2, C3, C6, B1, B6, WD \\
4  &                             All Types & 0.0044 & 0.0000 &      6 &      D6, B7, NW, NW, WD, WD &  D2, C3, C6, B1, B6, RD \\
5  &                             All Types & 0.0044 & 0.0044 &      6 &      D9, B7, NW, NW, RD, RD &  D2, C3, C6, B1, B9, WD \\
6  &                             All Types & 0.0044 & 

/tmp/ipykernel_3517/3161601294.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(


In [29]:
df = df.style.set_properties(subset="T9", **{"background-color": "green"})
# ).set_properties(
#     subset=predicted_observation, **{"background-color": "red", "color": "white"}
# )

In [30]:
s = ["<HTML>"]
s.append("<HEAD><TITLE>WorkSheet</TITLE></HEAD>")
s.append("<BODY>")
# s.append(df.to_html())
s.append(df.to_html())
s.append("</BODY></HTML>")
html = "".join(s)
html_file = open("case_study.html", "w")
html_file.write(html)
html_file.close()

In [34]:
# test1
-46128.1875/43582

-1.0584229154237987

In [35]:
#human
-8898.1074/6178

-1.4402893169310458

In [36]:
#7p
-2722.6028/11019

-0.24708256647608676